In [1]:
import os, pickle, time, re, sys, operator

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.metrics import auc, classification_report, roc_auc_score, f1_score, matthews_corrcoef, balanced_accuracy_score, r2_score , confusion_matrix, precision_score, recall_score

import pandas as pd
import numpy as np
import seaborn as sns

from my_util import *
from lime.lime.lime_tabular import LimeTabularExplainer

# from pyexplainer.pyexplainer_pyexplainer import PyExplainer
import matplotlib.pyplot as plt


from datetime import datetime
from collections import Counter


sys.path.append(os.path.abspath('../'))
from pyexplainer.pyexplainer_pyexplainer import PyExplainer

from IPython.display import display

In [2]:
data_path = './dataset/'
result_dir = './eval_result/'
dump_dataframe_dir = './dump_df/'
pyExp_dir = './pyExplainer_obj/'
other_object_dir = './other_object/'
# proj_name = 'qt' # ['openstack','qt']

In [3]:
# def test_lime(proj_name):
#     global_model, correctly_predict_df, indep, dep, feature_df = prepare_data_for_testing(proj_name)
#     all_eval_result = pd.DataFrame()
    
#     for i in range(0,len(feature_df)):
#         X_explain = feature_df.iloc[[i]]

#         row_index = str(X_explain.index[0])

#         py_exp = pickle.load(open(pyExp_dir+proj_name+'_rulefit_crossoverinterpolation_'+row_index+'.pkl','rb'))
#         lime_exp = pickle.load(open(pyExp_dir+proj_name+'_lime_'+row_index+'.pkl','rb'))

#         # this data can be used for both local and global model
#         py_exp_synthetic_data = py_exp['synthetic_data'].values
#         # this data can be used with global model only
#         lime_exp_synthetic_data = lime_exp['synthetic_instance_for_global_model']
#         # this data can be used with local model only
#         lime_exp_synthetic_data_local = lime_exp['synthetic_instance_for_lobal_model']
        
#         display(X_explain)
#         display(lime_exp_synthetic_data[:5,:])
#         display(lime_exp_synthetic_data_local[:5,:])
        
#         break
        
# test_lime('openstack')

In [4]:
flip_sign_dict = {
    '<': '>=',
    '>': '<=',
    '=': '!=',
    '>=': '<',
    '<=': '>',
    '!=': '=='
}

'''
    input: rule (str)
'''
def flip_rule(rule):
    rule = re.sub(r'\b=\b',' = ',rule) # for LIME
#     rule = rule.replace('&','and') # for RuleFit
    found_rule = re.findall('.* <=? [a-zA-Z]+ <=? .*', rule) # for LIME
    ret = ''
    
    # for LIME that has condition like this: 0.53 < nref <= 0.83
    if len(found_rule) > 0:
        found_rule = found_rule[0]
    
        var_in_rule = re.findall('[a-zA-Z]+',found_rule)

        var_in_rule = var_in_rule[0]
        
        splitted_rule = found_rule.split(var_in_rule)
        splitted_rule[0] = splitted_rule[0] + var_in_rule # for left side
        splitted_rule[1] = var_in_rule + splitted_rule[1] # for right side
        combined_rule = splitted_rule[0] + ' or ' + splitted_rule[1]
        ret = flip_rule(combined_rule)
        
    else:
        for tok in rule.split():
            if tok in flip_sign_dict:
                ret = ret + flip_sign_dict[tok] + ' '
            else:
                ret = ret + tok + ' '
    return ret

def get_top_k_global_features(global_model, indep, top_k_global_feature_num = 5):
    global_feature_df = pd.DataFrame()
    global_feature_df['feature'] = indep
    global_feature_df['importance'] = global_model.feature_importances_

    global_feature_df = global_feature_df.sort_values(by='importance',ascending=False)

    top_k_global_features = list(global_feature_df['feature'])[:top_k_global_feature_num]

    return top_k_global_features
    
def sort_global_feature(global_model, indep):
    global_feature_df = pd.DataFrame()
    global_feature_df['feature'] = indep
    global_feature_df['importance'] = global_model.feature_importances_

    global_feature_df = global_feature_df.sort_values(by='importance',ascending=False)

    sorted_global_features = list(global_feature_df['feature'])

    return sorted_global_features

def get_rule_str_of_rulefit(local_rulefit_model):
    rule_df = local_rulefit_model.get_rules()
#     print(rule_df)
    top_k = 5
    top_k_positive_rules = rule_df[(rule_df.coef > 0) & (rule_df.type=='rule')].sort_values("importance", ascending=False).head(top_k)
#     top_k_positive_rules = rule_df[(rule_df.coef > 0) & (rule_df.type=='rule')].sort_values("coef", ascending=False).head(top_k)

    the_best_defective_rule_str = list(top_k_positive_rules['rule'])[0]
    
    return the_best_defective_rule_str

def get_rule_str_of_rulefit_new_version(local_rulefit_model):
    rule_df = local_rulefit_model.get_rules()
    rule_df =  rule_df[(rule_df.coef > 0) & (rule_df.type=='rule')].sort_values("importance", ascending=False)
    
    rule_list = list(rule_df['rule'])
    dup_feature_in_rule = [] # true or false...
    
    for r in rule_list:
        var_in_rule = re.findall('[a-zA-Z]+', r)
        var_count = Counter(var_in_rule)
        max_count = max(list(var_count.values()))
        
        if max_count > 1:
            dup_feature_in_rule.append(True)
        else:
            dup_feature_in_rule.append(False)
           
    if False not in set(dup_feature_in_rule):
#         print('wtf')
        rule_df = rule_df.head(5)
        the_best_defective_rule_str = list(rule_df['rule'])[0]
        
    else:
        rule_df['contain_dup_var'] = dup_feature_in_rule    
        the_best_defective_rule_str = rule_df[rule_df['contain_dup_var']==False].iloc[0]['rule']
    
    return the_best_defective_rule_str

def aggregate_list(l):
    return np.mean(l), np.median(l)


In [5]:
# def test_new_rule_from_rulefit(proj_name):
#     global_model, correctly_predict_df, indep, dep, feature_df = prepare_data_for_testing(proj_name)
#     all_eval_result = pd.DataFrame()
    
#     c = 0
    
# #     py_exp_all_rules = []
# #     lime_all_rules = []
#     py_exp_all_vars = []
#     lime_all_vars = []
    
#     print('global feature feature importance ranking:')
#     print(sort_global_feature(global_model, indep))
#     for i in range(0,len(feature_df)):
#         X_explain = feature_df.iloc[[i]]

#         row_index = str(X_explain.index[0])

#         py_exp = pickle.load(open(pyExp_dir+proj_name+'_rulefit_crossoverinterpolation_'+row_index+'_20_rules.pkl','rb'))
#         py_exp_local_model = py_exp['local_model']
        
#         lime_exp = pickle.load(open(pyExp_dir+proj_name+'_lime_'+row_index+'.pkl','rb'))
        
# #         py_exp_rule = get_rule_str_of_rulefit(py_exp_local_model)
#         py_exp_rule_new = get_rule_str_of_rulefit_new_version(py_exp_local_model)
#         lime_the_best_defective_rule_str = lime_exp['rule'].as_list()[0][0]

#         py_exp_pred = eval_rule(py_exp_rule_new, X_explain)[0]
#         lime_pred = eval_rule(lime_the_best_defective_rule_str, X_explain)[0]

#         if py_exp_pred == 1:
#             py_exp_var_in_rule = list(set(re.findall('[a-zA-Z]+', py_exp_rule_new)))
#             py_exp_all_vars.extend(py_exp_var_in_rule)
#         if lime_pred == 1:
#             lime_var_in_rule = list(set(re.findall('[a-zA-Z]+', lime_the_best_defective_rule_str)))
#             lime_all_vars.extend(lime_var_in_rule)
            
# #         py_exp_all_rules.append(py_exp_rule_new)
# #         lime_all_rules.append(lime_the_best_defective_rule_str)
        
# #         eval_result = eval_rule(lime_the_best_defective_rule_str, X_explain)

# #         if eval_result[0]:
# #             c =c+1

    
# #     print(len(set(py_exp_all_rules)))
# #     print(len(set(lime_all_rules)))
    
#     print('pyExplainer var count')
#     print(Counter(py_exp_all_vars))
#     print('-'*100)
#     print('LIME var count')
#     print(Counter(lime_all_vars))
    
# print('openstack')
# test_new_rule_from_rulefit('openstack')
# print('*'*100)
# print('qt')
# test_new_rule_from_rulefit('qt')

In [6]:
def is_in_top_k_global_features(top_k_global_features, the_best_defective_rule_str):
    # remove numeric value
    new_the_best_defective_rule_str = re.sub('\d+','', the_best_defective_rule_str)

    # remove special characters
    new_the_best_defective_rule_str = re.sub('\W+',' ',new_the_best_defective_rule_str)
    splitted_rule = new_the_best_defective_rule_str.split()

    local_feature_count = 0
    
    found_features = set(splitted_rule).intersection(top_k_global_features)
    return list(found_features)

# def eval_rule(rule, X_explain):
#     var_in_rule = re.findall('[a-zA-Z]+',rule)
#     rule = rule.replace('&','and') # just for rulefit
#     rule = re.sub(r'\b=\b','==',rule)
# #             rule = rule.replace('=','==')

#     var_dict = {}

#     for var in var_in_rule:
#         var_dict[var] = float(X_explain[var])

#     eval_result = eval(rule,var_dict)
#     return eval_result

        
def prepare_data_for_testing(proj_name, global_model_name = 'RF'):
    global_model_name = global_model_name.upper()
    global_model = pickle.load(open(proj_name+'_'+global_model_name+'_global_model.pkl','rb'))

    correctly_predict_df = pd.read_csv(dump_dataframe_dir+proj_name+'_'+global_model_name+'_correctly_predict_as_defective.csv')
    correctly_predict_df = correctly_predict_df.set_index('commit_id')

    dep = 'defect'
    indep = correctly_predict_df.columns[:-3] # exclude the last 3 columns

    feature_df = correctly_predict_df.loc[:, indep]
    
    return global_model, correctly_predict_df, indep, dep, feature_df
    

# Global model evaluation

In [7]:
def get_prediction_result_df(proj_name, global_model_name):
    global_model_name = global_model_name.upper()
    if global_model_name not in ['RF','LR']:
        print('wrong global model name. the global model name must be RF or LR')
        return
    
    prediction_df_dir = dump_dataframe_dir+proj_name+'_'+global_model_name+'_prediction_result.csv'
    correctly_predict_df_dir = dump_dataframe_dir+proj_name+'_'+global_model_name+'_correctly_predict_as_defective.csv'
    
    if not os.path.exists(prediction_df_dir) or not os.path.exists(correctly_predict_df_dir):
        global_model = pickle.load(open(proj_name+'_'+global_model_name+'_global_model.pkl','rb'))

        pred = global_model.predict(x_test)
        defective_prob = global_model.predict_proba(x_test)[:,1]

        prediction_df = x_test.copy()
        prediction_df['pred'] = pred
        prediction_df['defective_prob'] = defective_prob
        prediction_df['defect'] = y_test

    #     print('AUC is',roc_auc_score(y_test, defective_prob))
        correctly_predict_df = prediction_df[(prediction_df['pred']==1) & (prediction_df['defect']==1)]

        print('total correct prediction: {}'.format(str(len(correctly_predict_df))))

        prediction_df.to_csv(prediction_df_dir)
        correctly_predict_df.to_csv(correctly_predict_df_dir)
    
    else:
        prediction_df = pd.read_csv(prediction_df_dir)
        correctly_predict_df = pd.read_csv(correctly_predict_df_dir)
        
        prediction_df = prediction_df.set_index('commit_id')
        correctly_predict_df = correctly_predict_df.set_index('commit_id')
        print('total correct prediction: {}'.format(str(len(correctly_predict_df))))
        
    return prediction_df, correctly_predict_df

def get_recall_at_k_percent_effort(percent_effort, result_df_arg, real_buggy_commits):
    cum_LOC_k_percent = (percent_effort/100)*result_df_arg.iloc[-1]['cum_LOC']
    buggy_line_k_percent =  result_df_arg[result_df_arg['cum_LOC'] <= cum_LOC_k_percent]
    buggy_commit = buggy_line_k_percent[buggy_line_k_percent['defect']==True]
    recall_k_percent_effort = len(buggy_commit)/float(len(real_buggy_commits))
    
    return recall_k_percent_effort

def eval_global_model(proj_name, prediction_df):
    ## since ld metric in openstack is removed by using autospearman, so this code is needed
    ## but this is not problem for qt
    
    if proj_name == 'openstack':
        x_train_original, x_test_original = prepare_data_all_metrics(proj_name, mode='all')
        prediction_df = prediction_df.copy()
#         print('add ld')
#         display(x_test_original['ld'])
        prediction_df['ld'] = list(x_test_original['ld'])
        
    prediction_df = prediction_df[['la','ld', 'pred', 'defective_prob' ,'defect']]
    prediction_df['LOC'] = prediction_df['la']+prediction_df['ld']
    
    
#     result_df['defect_density'] = result_df['defective_commit_prob']/result_df['LOC']
    prediction_df['defect_density'] = prediction_df['defective_prob']/prediction_df['LOC']
    prediction_df['actual_defect_density'] = prediction_df['defect']/prediction_df['LOC'] #defect density
    
    prediction_df = prediction_df.fillna(0)
    prediction_df = prediction_df.replace(np.inf, 0)
    
    prediction_df = prediction_df.sort_values(by='defect_density',ascending=False)
#     display(prediction_df.head())
#     display(np.sum(prediction_df[prediction_df['la']==0]['defect']))
    
    actual_result_df = prediction_df.sort_values(by='actual_defect_density',ascending=False)
    actual_worst_result_df = prediction_df.sort_values(by='actual_defect_density',ascending=True)

    prediction_df['cum_LOC'] = prediction_df['LOC'].cumsum()
    actual_result_df['cum_LOC'] = actual_result_df['LOC'].cumsum()
    actual_worst_result_df['cum_LOC'] = actual_worst_result_df['LOC'].cumsum()

    real_buggy_commits = prediction_df[prediction_df['defect'] == True]
    
#     display(prediction_df)
#     display(real_buggy_commits)
    
    
    AUC = roc_auc_score(prediction_df['defect'], prediction_df['defective_prob'])
    f1 = f1_score(prediction_df['defect'], prediction_df['pred'])
    
    ifa = real_buggy_commits.iloc[0]['cum_LOC']
#     print('ifa:',ifa)

    cum_LOC_20_percent = 0.2*prediction_df.iloc[-1]['cum_LOC']
    buggy_line_20_percent = prediction_df[prediction_df['cum_LOC'] <= cum_LOC_20_percent]
    buggy_commit = buggy_line_20_percent[buggy_line_20_percent['defect']==True]
    recall_20_percent_effort = len(buggy_commit)/float(len(real_buggy_commits))
    
    # find P_opt
    percent_effort_list = []
    predicted_recall_at_percent_effort_list = []
    actual_recall_at_percent_effort_list = []
    actual_worst_recall_at_percent_effort_list = []
    
    for percent_effort in np.arange(10,101,10):
        predicted_recall_k_percent_effort = get_recall_at_k_percent_effort(percent_effort, prediction_df, real_buggy_commits)
        actual_recall_k_percent_effort = get_recall_at_k_percent_effort(percent_effort, actual_result_df, real_buggy_commits)
        actual_worst_recall_k_percent_effort = get_recall_at_k_percent_effort(percent_effort, actual_worst_result_df, real_buggy_commits)
        
        percent_effort_list.append(percent_effort/100)
        
        predicted_recall_at_percent_effort_list.append(predicted_recall_k_percent_effort)
        actual_recall_at_percent_effort_list.append(actual_recall_k_percent_effort)
        actual_worst_recall_at_percent_effort_list.append(actual_worst_recall_k_percent_effort)

    p_opt = 1 - ((auc(percent_effort_list, actual_recall_at_percent_effort_list) - 
                 auc(percent_effort_list, predicted_recall_at_percent_effort_list)) /
                (auc(percent_effort_list, actual_recall_at_percent_effort_list) -
                auc(percent_effort_list, actual_worst_recall_at_percent_effort_list)))

    print('AUC: {}, F1: {}, IFA: {}, Recall@20%Effort: {}, Popt: {}'.format(AUC,f1,ifa,recall_20_percent_effort,p_opt))
    print(classification_report(prediction_df['defect'], prediction_df['pred']))
#     display(cum_LOC_20_percent)
#     display(buggy_line_20_percent)
# #     display(cum_LOC_20_percent)
#     display(buggy_commit)
#     print(len(real_buggy_commits))
#     display(recall_20_percent_effort)

def get_global_model_evaluation_result(proj_name):
    print('RF global model result')
    rf_prediction_df, rf_correctly_predict_df = get_prediction_result_df(proj_name, 'rf')
    eval_global_model(proj_name, rf_prediction_df)

    print('-'*100)
    
    print('LR global model result')
    lr_prediction_df, lr_correctly_predict_df = get_prediction_result_df(proj_name, 'lr')
    eval_global_model(proj_name, lr_prediction_df)

# RQ3 evaluation

In [10]:
def eval_rule(rule, x_df):
    var_in_rule = list(set(re.findall('[a-zA-Z]+', rule)))
    
    rule = re.sub(r'\b=\b','==',rule)
    if 'or' in var_in_rule:
        var_in_rule.remove('or')
        
    rule = rule.replace('&','and')
    
    eval_result_list = []
    
#     print(rule)

    for i in range(0,len(x_df)):
        x = x_df.iloc[[i]]
        col = x.columns
        var_dict = {}

        for var in var_in_rule:
            var_dict[var] = float(x[var])

#         print(var_dict)
        
        # if the rule does not satisfy clean commit, the truth value of the inversed rule when applied to clean commit is true
        eval_result = eval(rule,var_dict)
        eval_result_list.append(eval_result)
        
#         print(eval_result)
#         break
        
    return eval_result_list

# def summarize_rule_eval_result(py_exp_rule_str, lime_rule_str, x_df, ground_truth):
# #     print('Rulefit')
#     py_exp_all_eval_result = eval_rule(py_exp_rule_str, x_df)
# #     print('LIME')
#     lime_all_eval_result = eval_rule(lime_rule_str, x_df)

# #     print(py_exp_rule_str)
# #     print(lime_rule_str)
    
# #     tmp_df = x_df.copy()
# #     tmp_df['ground_truth'] = ground_truth
# #     tmp_df_clean = tmp_df[tmp_df['ground_truth']==False]
    
# #     display(tmp_df_clean)
    
#     py_exp_result_df = pd.DataFrame()
#     py_exp_result_df['ground_truth'] = ground_truth
#     py_exp_result_df['rule_result'] = py_exp_all_eval_result
#     py_exp_result_df = py_exp_result_df[py_exp_result_df['rule_result']==True] # get commit that matches counter rule
    
# #     print('py_exp_result_df')
# #     display(py_exp_result_df[py_exp_result_df['ground_truth']==False])
# #     print(len(py_exp_result_df))
#     # find ratio of clean commit
#     py_exp_satisfy_rule_ratio = 100*(len(py_exp_result_df[py_exp_result_df['ground_truth']==False])/len(py_exp_result_df)) if len(py_exp_result_df) > 0 else 0
    
#     lime_result_df = pd.DataFrame()
#     lime_result_df['ground_truth'] = ground_truth
#     lime_result_df['rule_result'] = lime_all_eval_result
    
#     lime_result_df = lime_result_df[lime_result_df['rule_result']==True] # get commit that matches counter rule
    
# #     print(len(lime_result_df))
    
#     # find ratio of clean commit
# #     print('lime_result_df')
# #     display(lime_result_df[lime_result_df['ground_truth']==False])
#     lime_satisfy_rule_ratio = 100*(len(lime_result_df[lime_result_df['ground_truth']==False])/len(lime_result_df))  if len(lime_result_df) > 0 else 0
    
# #     print(len(py_exp_result_df[py_exp_result_df['ground_truth']==False]))
# #     print(len(lime_result_df[lime_result_df['ground_truth']==False]))
    
#     return py_exp_satisfy_rule_ratio, lime_satisfy_rule_ratio

def summarize_rule_eval_result(rule_str, x_df):
#     print('Rulefit')
    all_eval_result = eval_rule(rule_str, x_df)
    all_eval_result = np.array(all_eval_result).astype(bool)
    
#     result_df = pd.DataFrame()
#     result_df['ground_truth'] = ground_truth
#     result_df['rule_result'] = all_eval_result
#     result_df = result_df[result_df['rule_result']==True] # get commit that matches counter rule
    
#     print('py_exp_result_df')
#     display(py_exp_result_df[py_exp_result_df['ground_truth']==False])
#     print(len(py_exp_result_df))
    # find ratio of clean commit
#     satisfy_rule_ratio = 100*(len(result_df[result_df['ground_truth']==False])/len(result_df)) if len(result_df) > 0 else 0

    return all_eval_result

'''
    input:
        local_model: local model of RuleFit
        X_explain: an instance to be explained
        
    return:
        g2_guide, g4_guide (string)
        more info of g2/g4 guidance refers to SQAPlanner paper
'''

def get_g2_g4_guidance(local_model, X_explain):
    rules = local_model.get_rules()
    rules = rules[(rules['type']=='rule') & (rules['coef'] < 0) & (rules['importance'] > 0)]
    rules_list = list(rules['rule'])
    
    rule_eval_result = []

    for r in rules_list:
        py_exp_pred = eval_rule(r, X_explain)[0]
        rule_eval_result.append(py_exp_pred)
        
    rules['is_satisfy_instance'] = rule_eval_result
    
    g2_guide_df = rules[rules['is_satisfy_instance']==True]
    g4_guide_df = rules[rules['is_satisfy_instance']==False]

    g2_guide_df = g2_guide_df.sort_values(by='importance', ascending=False)
    g4_guide_df = g4_guide_df.sort_values(by='importance', ascending=False)
    
    g2_guide = g2_guide_df.iloc[0]['rule']
    g4_guide = g4_guide_df.iloc[0]['rule']
    
    return g2_guide, g4_guide

def get_g1_guidance(local_model, X_explain):
    rules = local_model.get_rules()
    rules = rules[(rules['type']=='rule') & (rules['coef'] > 0) & (rules['importance'] > 0)]
    rules_list = list(rules['rule'])
    
    rule_eval_result = []

    for r in rules_list:
        py_exp_pred = eval_rule(r, X_explain)[0]
        rule_eval_result.append(py_exp_pred)
        
    rules['is_satisfy_instance'] = rule_eval_result
    
    g1_guide_df = rules[rules['is_satisfy_instance']==True]

    g1_guide_df = g1_guide_df.sort_values(by='importance', ascending=False)
    
    g1_guide = g1_guide_df.iloc[0]['rule']
    
    return g1_guide
def get_g1_guidancex(local_model, X_explain):
    rules = local_model.get_rules()
    rules = rules[(rules['type']=='rule') & (rules['coef'] > 0) & (rules['importance'] > 0)]
    rules_list = list(rules['rule'])
    
    rule_eval_result = []

    for r in rules_list:
        py_exp_pred = eval_rule(r, X_explain)[0]
        rule_eval_result.append(py_exp_pred)
        
    rules['is_satisfy_instance'] = rule_eval_result
    
    g1_guide_df = rules[rules['is_satisfy_instance']==True]

    g1_guide_df = g1_guide_df.sort_values(by='coef', ascending=False)
    
    g1_guide = g1_guide_df.iloc[0]['rule']
    
    return g1_guide

In [11]:
def test_rule(proj_name):
    global_model, correctly_predict_df, indep, dep, feature_df = prepare_data_for_testing(proj_namez)
    x_test, y_test = prepare_data(proj_name, mode = 'test')
    
    rq3_eval_result = pd.DataFrame() # for train data

    py_exp_guide = []
    lime_guide = []
    
    for i in range(0,len(feature_df)):
        X_explain = feature_df.iloc[[i]]

        row_index = str(X_explain.index[0])

        py_exp = pickle.load(open(pyExp_dir+proj_name+'_rulefit_crossoverinterpolation_'+row_index+'.pkl','rb'))
        lime_exp = pickle.load(open(pyExp_dir+proj_name+'_lime_'+row_index+'.pkl','rb'))

        py_exp_local_model = py_exp['local_model']
        lime_exp_local_model = lime_exp['local_model']
        
        py_exp_the_best_defective_rule_str = get_rule_str_of_rulefit(py_exp_local_model)
#         print(py_exp_the_best_defective_rule_str)
        
        total_cond_in_py_exp = len(py_exp_the_best_defective_rule_str.split('&'))
        
        lime_the_best_defective_rule_str = lime_exp['rule'].as_list()[0][0]

        py_exp_pred = eval_rule(py_exp_the_best_defective_rule_str, X_explain)[0]
        lime_pred = eval_rule(lime_the_best_defective_rule_str, X_explain)[0]

#         print(py_exp_pred, lime_pred)
        if py_exp_pred:
            py_exp_the_best_defective_rule_str = flip_rule(py_exp_the_best_defective_rule_str)
            py_exp_guide.append(py_exp_the_best_defective_rule_str)

        if lime_pred:
            lime_the_best_defective_rule_str = flip_rule(lime_the_best_defective_rule_str)
            lime_guide.append(lime_the_best_defective_rule_str)
        
    print(set(py_exp_guide))
    print('total guidance:',len(set(py_exp_guide)))
    print('-'*100)
    print(set(lime_guide))
    print('total guidance:',len(set(lime_guide)))

In [12]:
def eval_PyExplainer_guidance(proj_name, global_model, method_name, row_index, guidance, x_test, y_test_flip , flip=False):
    guidance_list = guidance.split('&')
    
    guide_eval_result = pd.DataFrame()
    
    for condition in guidance_list:
        if flip:
            condition = flip_rule(condition)
            
        py_exp_guidance_eval = summarize_rule_eval_result(condition, x_test)

        guide_prec = precision_score(y_test_flip, py_exp_guidance_eval)
        guide_rec = recall_score(y_test_flip, py_exp_guidance_eval)

        py_exp_serie_test = pd.Series(data=[proj_name, row_index, method_name, global_model,condition, guide_prec, guide_rec])
        guide_eval_result = guide_eval_result.append(py_exp_serie_test,ignore_index=True)
        
    return guide_eval_result

In [13]:


def rq3_eval(proj_name, global_model_name):
    global_model, correctly_predict_df, indep, dep, feature_df = prepare_data_for_testing(proj_name, global_model_name)
    x_test, y_test = prepare_data(proj_name, mode = 'test')
    
    y_test_flip = [False if val else True for val in y_test]
    
    rq3_explanation_result = pd.DataFrame()
    
    pyexp_guidance_result_list = []
    lime_guidance_result_df = pd.DataFrame()
    
    for i in range(0,len(feature_df)):

        X_explain = feature_df.iloc[[i]]

        row_index = str(X_explain.index[0])

        exp_obj = pickle.load(open(pyExp_dir+proj_name+'_'+global_model_name+'_all_explainer_'+row_index+'.pkl','rb'))
        py_exp = exp_obj['pyExplainer']
        lime_exp = exp_obj['LIME']

        # load local models
        py_exp_local_model = py_exp['local_model']
        lime_exp_local_model = lime_exp['local_model']
        
        # generate explanations                
        py_exp_the_best_defective_rule_str = get_g1_guidance(py_exp_local_model, X_explain)
        lime_the_best_defective_rule_str = lime_exp['rule'].as_list()[0][0]

        # check whether explanations apply to the instance to be explained
        py_exp_pred = eval_rule(py_exp_the_best_defective_rule_str, X_explain)[0]
        lime_pred = eval_rule(lime_the_best_defective_rule_str, X_explain)[0]

        # split rule to get list of conditions
        condition_list = py_exp_the_best_defective_rule_str.split('&')

        # for explanations
        for condition in condition_list:
            condition = condition.strip()

            py_exp_rule_eval = summarize_rule_eval_result(condition, x_test)

            rule_prec = precision_score(y_test, py_exp_rule_eval)
            rule_rec = recall_score(y_test, py_exp_rule_eval)

            py_exp_serie_test = pd.Series(data=[proj_name, row_index, 'pyExplainer',global_model_name, condition, rule_prec, rule_rec, py_exp_pred])
            rq3_explanation_result = rq3_explanation_result.append(py_exp_serie_test,ignore_index=True)

        # for guidance
        g2_guide, g4_guide = get_g2_g4_guidance(py_exp_local_model, X_explain)

        flip_guide_eval_result = eval_PyExplainer_guidance(proj_name, global_model_name,'PyExpFlip', row_index,
                                                           py_exp_the_best_defective_rule_str, x_test, y_test_flip, flip=True)
        g2_guide_eval_result = eval_PyExplainer_guidance(proj_name, global_model_name,'PyExpG2', row_index,
                                                           g2_guide, x_test, y_test_flip, flip=False)
        g4_guide_eval_result = eval_PyExplainer_guidance(proj_name, global_model_name,'PyExpG4', row_index,
                                                           g4_guide, x_test, y_test_flip, flip=False)

        pyexp_guidance_result_list.append(flip_guide_eval_result)
        pyexp_guidance_result_list.append(g2_guide_eval_result)
        pyexp_guidance_result_list.append(g4_guide_eval_result)
        # PyExp END
        
       
        # LIME START
        lime_rule_eval = summarize_rule_eval_result(lime_the_best_defective_rule_str, x_test)

        rule_prec = precision_score(y_test, lime_rule_eval)
        rule_rec = recall_score(y_test, lime_rule_eval)

        lime_serie_test = pd.Series(data=[proj_name, row_index, 'LIME',global_model_name, lime_the_best_defective_rule_str, rule_prec, rule_rec, lime_pred])
        rq3_explanation_result = rq3_explanation_result.append(lime_serie_test,ignore_index=True)

        lime_guidance = flip_rule(lime_the_best_defective_rule_str)
        lime_guidance_eval = summarize_rule_eval_result(lime_guidance, x_test)
#             tn, fp, fn, tp = confusion_matrix(y_test, lime_rule_eval, labels=[1,0]).ravel()
#             tp_rate = tp/(tp+fn)
#             tn_rate = tn/(tn+fp)

        guide_prec = precision_score(y_test_flip, lime_guidance_eval)
        guide_rec = recall_score(y_test_flip, lime_guidance_eval)

        lime_serie_test = pd.Series(data=[proj_name, row_index, 'LIME', global_model_name, lime_guidance, guide_prec, guide_rec])
        lime_guidance_result_df = lime_guidance_result_df.append(lime_serie_test, ignore_index=True)
            
        print('finished {} from {} commits'.format(str(i+1),len(feature_df)))
        
        
    pyexp_guidance_result_df = pd.concat(pyexp_guidance_result_list)
    
    rq3_guidance_result = pd.concat([pyexp_guidance_result_df, lime_guidance_result_df])
    
    
    
    rq3_explanation_result.columns = ['project','commit_id','method','global_model','explanation','precision','recall', 'isSatisfy']
    rq3_guidance_result.columns = ['project','commit_id','method', 'global_model','guidance','precision','recall']

    rq3_explanation_result.to_csv(result_dir+'RQ3_'+proj_name+'_'+global_model_name+'_explanation_eval_split_rulefit_condition.csv',index=False)
    rq3_guidance_result.to_csv(result_dir+'RQ3_'+proj_name+'_'+global_model_name+'_guidance_eval_split_rulefit_condition.csv',index=False)


In [14]:
proj_name = 'openstack'
global_model_name = 'RF'

global_model, correctly_predict_df, indep, dep, feature_df = prepare_data_for_testing(proj_name, global_model_name)
x_test, y_test = prepare_data(proj_name, mode = 'test')

y_test_flip = [False if val else True for val in y_test]

rq3_explanation_result = pd.DataFrame()

pyexp_guidance_result_list = []
lime_guidance_result_df = pd.DataFrame()


In [16]:
i=0
X_explain = feature_df.iloc[[i]]

row_index = str(X_explain.index[0])

exp_obj = pickle.load(open(pyExp_dir+proj_name+'_'+global_model_name+'_all_explainer_'+row_index+'.pkl','rb'))
py_exp = exp_obj['pyExplainer']
lime_exp = exp_obj['LIME']

py_exp_local_model = py_exp['local_model']
lime_exp_local_model = lime_exp['local_model']

py_exp_the_best_defective_rule_str = get_rule_str_of_rulefit(py_exp_local_model)
lime_the_best_defective_rule_str = lime_exp['rule'].as_list()[0][0]

py_exp_pred = eval_rule(py_exp_the_best_defective_rule_str, X_explain)[0]
lime_pred = eval_rule(lime_the_best_defective_rule_str, X_explain)[0]


In [17]:
lime_exp['rule'].as_list()[0][0]

'la > 58.00'

In [20]:
def get_g1_guidance(local_model, X_explain):
    rules = local_model.get_rules()
    rules = rules[(rules['type']=='rule') & (rules['coef'] > 0) & (rules['importance'] > 0)]
    rules_list = list(rules['rule'])
    
    rule_eval_result = []

    for r in rules_list:
        py_exp_pred = eval_rule(r, X_explain)[0]
        rule_eval_result.append(py_exp_pred)
        
    rules['is_satisfy_instance'] = rule_eval_result
    
    g1_guide_df = rules[rules['is_satisfy_instance']==True]

    g1_guide_df = g1_guide_df.sort_values(by='importance', ascending=False)
    
    g1_guide = g1_guide_df.iloc[0]['rule']
    
    return g1_guide

g1_guide = get_g1_guidance(py_exp_local_model, X_explain)
g1_guidex = get_g1_guidancex(py_exp_local_model, X_explain)

In [24]:
rules = py_exp_local_model.get_rules()
rules = rules[(rules['type']=='rule') & (rules['coef'] > 0) & (rules['importance'] > 0)]
rules_list = list(rules['rule'])

rule_eval_result = []

for r in rules_list:
    py_exp_pred = eval_rule(r, X_explain)[0]
    rule_eval_result.append(py_exp_pred)

rules['is_satisfy_instance'] = rule_eval_result

g1_guide_df = rules[rules['is_satisfy_instance']==True]

In [25]:
g1_guide_df = g1_guide_df.sort_values(by='importance', ascending=False)
g1_guide_df

,rule,type,coef,support,importance,is_satisfy_instance
278,age <= 12.769999980926514 & age > 0.5099999904...,rule,0.062858,0.550914,0.031266,True
1685,ndev <= 30.90499973297119 & la > 11.2899999618...,rule,0.056020,0.480418,0.027988,True
771,age > 0.5250000059604645 & nrev <= 2.985000014...,rule,0.057506,0.344648,0.027330,True
1200,nd <= 2.4600000381469727 & ent <= 0.9549999833...,rule,0.055320,0.590078,0.027207,True
902,nrev > 1.0800000429153442 & nrev <= 2.97000002...,rule,0.055707,0.315927,0.025897,True
...,...,...,...,...,...,...
1605,age <= 15.349999904632568 & age > 1.8000000119...,rule,0.000499,0.543081,0.000248,True
1352,la > 48.1200008392334 & asawr <= 0.00499999988...,rule,0.000808,0.078329,0.000217,True
646,rsawr <= 0.13499999791383743,rule,0.000534,0.161880,0.000197,True
1030,asawr <= 0.004999999888241291 & la > 10.860000...,rule,0.000220,0.190601,0.000086,True


In [28]:
g1_guide_df = g1_guide_df.sort_values(by='support', ascending=False)
g1_guide_df

,rule,type,coef,support,importance,is_satisfy_instance
796,ndev <= 44.01500129699707 & ndev <= 49.2299995...,rule,0.003537,0.898172,0.001070,True
973,ndev <= 42.08500099182129,rule,0.013521,0.898172,0.004089,True
1704,nd > 1.5799999833106995 & rtime > -19.22000026...,rule,0.011864,0.819843,0.004560,True
754,nrev <= 6.670000076293945 & ent <= 0.995000004...,rule,0.000672,0.780679,0.000278,True
168,la > 3.1399999856948853 & asawr > -0.004999999...,rule,0.018433,0.775457,0.007692,True
...,...,...,...,...,...,...
273,la > 210.02000427246094 & app > 2.995000004768...,rule,0.020183,0.028721,0.003371,True
1672,la > 179.98500061035156 & app > 3.975000023841858,rule,0.002435,0.026110,0.000388,True
218,app > 2.990000009536743 & la > 241.5,rule,0.023011,0.023499,0.003486,True
1738,la > 109.65499877929688 & rsawr <= 0.100000001...,rule,0.002141,0.023499,0.000324,True


In [22]:
g1_guide

'age <= 12.769999980926514 & age > 0.5099999904632568 & la > 3.4800000190734863'

In [21]:
g1_guidex

'age <= 12.769999980926514 & age > 0.5099999904632568 & la > 3.4800000190734863'

In [83]:
len(y_test)

3963

In [66]:
x_test

,la,nd,ns,ent,nrev,rtime,self,ndev,age,app,rrexp,asawr,rsawr
commit_id,,,,,,,,,,,,,
5ce74eb5469b7f88f4448ccbc1afaa802a7cfdef,17,2,1,0.936667,7.0,17.869722,0,38.0,0.045706,1.0,1306.0,0.113107,0.247377
878ac164a391e761c72c5fdcd12f0caf48c7d359,49,9,3,0.718116,5.0,9.979109,0,116.0,13.035437,4.0,373.0,0.597853,0.296449
bbab55c4da531e4695a6e3e577aaa4975f0fce79,16,2,1,0.988699,6.0,81.107118,0,123.0,0.302407,7.0,2810.0,0.005334,0.389047
a2ba455e2d2bc41f4a80a08d5434b741ed715ef4,519,13,1,0.861772,24.0,16.003877,0,194.0,2.348328,3.0,2086.0,0.122311,0.389511
9382ee659212285a203550cf60476dd146d27a29,89,2,1,0.996276,3.0,105.237789,0,9.0,21.841916,3.0,632.0,0.235955,0.820225
...,...,...,...,...,...,...,...,...,...,...,...,...,...
209c09a76f5e90aaa2899804686e6a513703d887,2,1,1,0.000000,2.0,20.872523,1,0.0,0.000000,0.0,0.0,0.000000,0.000000
1fdc4afa157887d45e2f326d7373a5b1e8ee7aeb,180,39,1,0.846758,24.0,92.130301,0,411.0,0.501826,2.0,720.0,0.000268,0.094218
aaf5762be5d37cac022dc321b6400b9743a25303,4,2,1,0.413817,2.0,18.489861,0,49.0,26.872975,4.0,2299.0,0.004550,0.352067


In [53]:
y_test

commit_id
5ce74eb5469b7f88f4448ccbc1afaa802a7cfdef    False
878ac164a391e761c72c5fdcd12f0caf48c7d359    False
bbab55c4da531e4695a6e3e577aaa4975f0fce79    False
a2ba455e2d2bc41f4a80a08d5434b741ed715ef4    False
9382ee659212285a203550cf60476dd146d27a29    False
                                            ...  
209c09a76f5e90aaa2899804686e6a513703d887    False
1fdc4afa157887d45e2f326d7373a5b1e8ee7aeb    False
aaf5762be5d37cac022dc321b6400b9743a25303    False
75c7f6a17a5bb78074518877bf73f0071b7758eb    False
3824051b1e5618388a17c88867a3037397bc96b7    False
Name: defect, Length: 3963, dtype: bool

In [64]:
feature_df.iloc[[i]]

,la,nd,ns,ent,nrev,rtime,self,ndev,age,app,rrexp,asawr,rsawr
commit_id,,,,,,,,,,,,,
10593c2eaf4eff4edb13b70f023acbbf743f129f,186,9,1,0.757325,10.0,19.790243,0,161.0,2.372364,2.0,538.0,0.122838,0.093364


In [65]:
py_exp_pred

True

In [63]:

for i in range(0,len(feature_df)):
    X_explain = feature_df.iloc[[i]]

    row_index = str(X_explain.index[0])

    exp_obj = pickle.load(open(pyExp_dir+proj_name+'_'+global_model_name+'_all_explainer_'+row_index+'.pkl','rb'))
    py_exp = exp_obj['pyExplainer']
    lime_exp = exp_obj['LIME']

    py_exp_local_model = py_exp['local_model']
    lime_exp_local_model = lime_exp['local_model']

    py_exp_the_best_defective_rule_str = get_rule_str_of_rulefit(py_exp_local_model)
    lime_the_best_defective_rule_str = lime_exp['rule'].as_list()[0][0]

    py_exp_pred = eval_rule(py_exp_the_best_defective_rule_str, X_explain)[0]
    lime_pred = eval_rule(lime_the_best_defective_rule_str, X_explain)[0]
    
    print(i, py_exp_pred)
    if py_exp_pred:

        condition_list = py_exp_the_best_defective_rule_str.split('&')

        # for explanation
        for condition in condition_list:
            condition = condition.strip()

            py_exp_rule_eval = summarize_rule_eval_result(condition, x_test)

            rule_prec = precision_score(y_test, py_exp_rule_eval)
            rule_rec = recall_score(y_test, py_exp_rule_eval)

            py_exp_serie_test = pd.Series(data=[proj_name, row_index, 'pyExplainer',global_model_name, condition, rule_prec, rule_rec])
            rq3_explanation_result = rq3_explanation_result.append(py_exp_serie_test,ignore_index=True)

        # for guidance
        g2_guide, g4_guide = get_g2_g4_guidance(py_exp_local_model, X_explain)

        flip_guide_eval_result = eval_PyExplainer_guidance(proj_name, global_model_name,'PyExpFlip', row_index,
                                                           py_exp_the_best_defective_rule_str, x_test, y_test_flip, flip=True)
        g2_guide_eval_result = eval_PyExplainer_guidance(proj_name, global_model_name,'PyExpG2', row_index,
                                                           g2_guide, x_test, y_test_flip, flip=False)
        g4_guide_eval_result = eval_PyExplainer_guidance(proj_name, global_model_name,'PyExpG4', row_index,
                                                           g4_guide, x_test, y_test_flip, flip=False)

        pyexp_guidance_result_list.append(flip_guide_eval_result)
        pyexp_guidance_result_list.append(g2_guide_eval_result)
        pyexp_guidance_result_list.append(g4_guide_eval_result)

        break
    
    if i == 5:
        break
    if lime_pred:
        lime_rule_eval = summarize_rule_eval_result(lime_the_best_defective_rule_str, x_test)

        rule_prec = precision_score(y_test, lime_rule_eval)
        rule_rec = recall_score(y_test, lime_rule_eval)

        lime_serie_test = pd.Series(data=[proj_name, row_index, 'LIME',global_model_name, lime_the_best_defective_rule_str, rule_prec, rule_rec])
        rq3_explanation_result = rq3_explanation_result.append(lime_serie_test,ignore_index=True)

        lime_guidance = flip_rule(lime_the_best_defective_rule_str)
        lime_guidance_eval = summarize_rule_eval_result(lime_guidance, x_test)
#             tn, fp, fn, tp = confusion_matrix(y_test, lime_rule_eval, labels=[1,0]).ravel()
#             tp_rate = tp/(tp+fn)
#             tn_rate = tn/(tn+fp)

        guide_prec = precision_score(y_test_flip, lime_guidance_eval)
        guide_rec = recall_score(y_test_flip, lime_guidance_eval)

        lime_serie_test = pd.Series(data=[proj_name, row_index, 'LIME', global_model_name, lime_guidance, guide_prec, guide_rec])
        lime_guidance_result_df = lime_guidance_result_df.append(lime_serie_test, ignore_index=True)

    print('finished {} from {} commits'.format(str(i+1),len(feature_df)))


# pyexp_guidance_result_df = pd.concat(pyexp_guidance_result_list)

# rq3_guidance_result = pd.concat([pyexp_guidance_result_df, lime_guidance_result_df])



# rq3_explanation_result.columns = ['project','commit_id','method','global_model','explanation','precision','recall']
# rq3_guidance_result.columns = ['project','commit_id','method', 'global_model','guidance','precision','recall']

# rq3_explanation_result.to_csv(result_dir+'RQ3_'+proj_name+'_'+global_model_name+'_explanation_eval_split_rulefit_condition.csv',index=False)
# rq3_guidance_result.to_csv(result_dir+'RQ3_'+proj_name+'_'+global_model_name+'_guidance_eval_split_rulefit_condition.csv',index=False)


    #         break
        
#     rq3_eval_result.columns = ['project', 'commit id', 'method', 'explanation','guidance', 'explantion_precision','explantion_recall', 
#                               'guidance_precision','guidance_recall', 'explanation_true_negative', 'explanation_false_positive',
#                               'explanation_false_negative','explanation_true_positive']
    
#     rq3_eval_result.to_csv(result_dir+'RQ3_'+proj_name+'_'+global_model_name+'_split_rulefit_condition.csv',index=False)
#     print('finished RQ3 of',proj_name)
    
#     display(rq3_eval_result)
    

0 False
finished 1 from 198 commits
1 False
finished 2 from 198 commits
2 False
finished 3 from 198 commits
3 False
finished 4 from 198 commits
4 False
finished 5 from 198 commits
5 True


/home/oathaha/.conda/envs/env_oat/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [35]:
# rq3_eval('openstack','RF')

In [37]:
start = time.time()
print('openstack')
rq3_eval('openstack','RF')
end = time.time()
print('finished in',str(end-start), 'secs')

start = time.time()
print('qt')
rq3_eval('qt','RF')
end = time.time()
print('finished in',str(end-start), 'secs')

openstack
finished 1 from 198 commits
finished 2 from 198 commits
finished 3 from 198 commits
finished 4 from 198 commits
finished 5 from 198 commits


/home/oathaha/.conda/envs/env_oat/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


finished 6 from 198 commits
finished 7 from 198 commits


KeyboardInterrupt: 

In [38]:
start = time.time()
print('openstack')
rq3_eval('openstack','LR')
end = time.time()
print('finished in',str(end-start), 'secs')

start = time.time()
print('qt')
rq3_eval('qt','LR')
end = time.time()
print('finished in',str(end-start), 'secs')

openstack


/home/oathaha/.conda/envs/env_oat/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


KeyboardInterrupt: 